In [ ]:
from distributed import Client, LocalCluster

hello = LocalCluster(n_workers = 6, threads_per_worker = 1)
client = Client(hello)

client

In [ ]:
import pyximport; pyximport.install()
from cython_traffic_CA import run_simulation

def run_parallel(num_lanes, num_places, density, P_lc, time_steady, num_iters, R_hdv, 
                 max_platoon_size, N_c, visualise = False):
    pyximport.install()
    from cython_traffic_CA import run_simulation
    return run_simulation(num_lanes, num_places, density, P_lc, time_steady, num_iters, 
                          R_hdv, max_platoon_size, N_c, visualise)

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt

%matplotlib inline
%config InlineBackend.figure_format = 'svg' 

mpl.rcParams['font.family'] = 'serif'
mpl.rcParams['font.weight'] = 'ultralight'
mpl.rcParams['font.serif'] = 'Times New Roman'
mpl.rcParams['font.size'] = 12
mpl.rcParams['font.style'] = 'normal'
mpl.rcParams['mathtext.default'] = 'regular'

import numpy as np

def single_run_visualisation(num_lanes, num_places, density, P_lc, time_steady, num_iters, R_hdv, 
                             max_platoon_size, N_c, visualise = True):
    
    rho, flux, change_lane, v_av, episodes_I = run_parallel(num_lanes, num_places, density, P_lc, 
                                                            time_steady, num_iters, R_hdv, max_platoon_size, N_c,visualise)
    episodes = np.asarray(episodes_I)
    for lane in range(num_lanes):
        indicator = int('1' + str(num_lanes) + str(lane+1))
        plt.subplot(indicator)
        plt.title('Lane ' + str(lane+1))
        plt.xlabel('Lane site')
        plt.ylabel('Time step')
        plt.tight_layout()
        plt.imshow(episodes[lane], cmap="Greys", interpolation="None") #"None" "nearest"
    plt.savefig("spatiotemp.tiff", dpi=300, format="tiff")
    plt.show()

In [ ]:
single_run_visualisation(num_lanes = 1, num_places = 1000, density = 0.45, P_lc = 0.0, time_steady = 0, 
                         num_iters = 1000, R_hdv = 1.0, max_platoon_size = 0, N_c = 0, visualise = True)

In [ ]:
def fundamental_diagram_parallel(num_lanes, num_places, density_steps, P_lc, time_steady, time_iters, 
                                 r_hdv, max_platoon_size, R):
    '''
    Returns data (density, flux and lane change rate arrays) for drawing a fundamental diagram.
    num_lanes (int) - number of lanes in the road;
    num_places (int) - number of sites (or cells) in each lane;
    density_steps (int) - number of steps in density between [0,1];
    time_iters (int) -  number of time iterarions.
    '''
    rho_ar = np.zeros(density_steps)
    flux_ar = np.zeros(density_steps)
    change_lane = np.zeros(density_steps)
    v_av_ar = np.zeros(density_steps)

    density_range = np.linspace(0.0, 0.7, density_steps)
    
    # initialize arrays for parallel implementation
    num_lanes_r = np.full(density_steps, num_lanes, dtype = int)
    num_cells_r = np.full(density_steps, num_places, dtype = int)
    time_steady_r = np.full(density_steps, time_steady, dtype = int)
    time_iters_r = np.full(density_steps, time_iters, dtype = int)
    N_c_r = (R*density_range*num_lanes*num_places).astype(int)
    P_lc_r, r_hdv_r = np.full(density_steps, P_lc), np.full(density_steps, r_hdv)
    max_platoon_size_r = np.full(density_steps, max_platoon_size, dtype = int)
    
    futs = client.map(run_parallel, num_lanes_r, num_cells_r, density_range, P_lc_r, \
                      time_steady_r, time_iters_r, r_hdv_r, max_platoon_size_r, N_c_r, pure = False)
    results = client.gather(futs)
    
    for i in range(density_steps):
        rho_ar[i], flux_ar[i], change_lane[i], v_av_ar[i] = results[i]
    
    return rho_ar, flux_ar, change_lane, v_av_ar

def R_influence_parallel(num_lanes, num_places, P_lc, time_steady, time_iters, R_steps, density):
    '''
    Returns: R_range, flux_ar, v_av_ar
    num_lanes (int) - number of lanes in the road;
    num_places (int) - number of sites (or cells) in each lane;
    density (float from 0 to 1) - density of cars;
    time_iters (int) -  number of time iterarions.
    '''
    flux_ar = np.zeros(R_steps)
    v_av_ar = np.zeros(R_steps)
    
    R_range = np.linspace(0, 0.9, R_steps)
    N_c_r = (R_range*density*num_lanes*num_places).astype(int)
    
    # initialize arrays for parallel implementation
    num_lanes_r = np.full(R_steps, num_lanes, dtype = int)
    num_cells_r = np.full(R_steps, num_places, dtype = int)
    time_steady_r = np.full(R_steps, time_steady, dtype = int)
    time_iters_r = np.full(R_steps, time_iters, dtype = int)
    density_r = np.full(R_steps, density)
    P_lc_r = np.full(R_steps, P_lc)

    futs = client.map(run_parallel, num_lanes_r, num_cells_r, density_r, P_lc_r, \
                      time_steady_r, time_iters_r, N_c_r, pure = False)
    results = client.gather(futs)
    
    for i in range(R_steps):
        rho, flux_ar[i], change_lane, v_av_ar[i] = results[i]
        
    return R_range, flux_ar, v_av_ar

In [ ]:
def draw_fundamental_diagram(num_lanes, num_places, density_steps, P_lc, time_steady, time_iters, 
                             r_hdv_list, max_platoon_size, R):
    r, flux, CLR, v_av = [], [], [], []
    M = []
    k = 0
    print("Draw fundamental diagramm routine has begun:")
    for r_hdv in tqdm(r_hdv_list):
        r.append(None); flux.append(None); CLR.append(None); v_av.append(None)
        r[k], flux[k], CLR[k], v_av[k] = fundamental_diagram_parallel(num_lanes, num_places, density_steps, 
                                                                      P_lc, time_steady, time_iters,
                                                                      r_hdv, max_platoon_size, R)
        M.append(1.0 - flux[k]/(num_lanes*r[k]))
        k += 1
    
    
    single_plot(x = r, y = flux, var_list = r_hdv_list, var_name = 'HDV ratio', \
                title = 'Fundamental diagram', x_label = 'Normalized density', y_label = 'Normalized flow')
    single_plot(x = r, y = CLR, var_list = r_hdv_list, var_name = 'HDV ratio', \
                title = 'Lane changing rate', x_label = 'Normalized density', \
                y_label = 'Lane changing rate per cell')
    single_plot(x = r, y = v_av, var_list = r_hdv_list, var_name = 'HDV ratio', \
                title = 'Average velocity of conventional vehicle', x_label = 'Normalized density', \
                y_label = 'average velocity')
    single_plot(x = r, y = M, var_list = r_hdv_list, var_name = 'HDV ratio', \
                title = 'Order parameter', x_label = 'Normalized density', \
                y_label = 'Order parameter')    
    
def draw_fundamental_diagram_1(num_lanes, num_places, density_steps, P_lc, time_steady, time_iters, 
                             r_hdv, max_platoon_size_list, R):
    r, flux, CLR, v_av = [], [], [], []
    M = []
    k = 0
    print("Draw fundamental diagramm routine has begun:")
    for max_platoon_size in tqdm(max_platoon_size_list):
        r.append(None); flux.append(None); CLR.append(None); v_av.append(None)
        r[k], flux[k], CLR[k], v_av[k] = fundamental_diagram_parallel(num_lanes, num_places, density_steps, 
                                                                      P_lc, time_steady, time_iters,
                                                                      r_hdv, max_platoon_size, R)
        M.append(1.0 - flux[k]/(num_lanes*r[k]))
        r[k] = r[k]*1000.0/7.5
        k += 1
    
    
    single_plot(x = r, y = flux, var_list = max_platoon_size_list, var_name = 'Max platoon size', \
                title = 'Fundamental diagram', x_label = 'Density [Vehicle/km]', y_label = 'Flow [Vehicle/h]')
    single_plot(x = r, y = CLR, var_list = max_platoon_size_list, var_name = 'Max platoon size', \
                title = 'Lane changing rate', x_label = 'Normalized density', \
                y_label = 'Lane changing rate per cell')
    single_plot(x = r, y = v_av, var_list = max_platoon_size_list, var_name = 'Max platoon size', \
                title = 'Average velocity of conventional vehicle', x_label = 'Normalized density', \
                y_label = 'average velocity')
    single_plot(x = r, y = M, var_list = max_platoon_size_list, var_name = 'Max platoon size', \
                title = 'Order parameter', x_label = 'Normalized density', \
                y_label = 'Order parameter')  
    
def draw_R_inf(num_lanes, num_places, P_lc, time_steady, time_iters, R_steps, density_list):
    R, flux, v_av = [], [], []
    k = 0
    print("R impact diagramm routine has begun:")
    for density in tqdm(density_list):
        R.append(None); flux.append(None); v_av.append(None)
        R[k], flux[k], v_av[k] = R_influence_parallel(num_lanes, num_places, P_lc, time_steady, \
                                             time_iters, R_steps, density)
        k += 1
    
    single_plot(x = R, y = flux, var_list = density_list, var_name = 'density', \
                title = 'R influence', x_label = 'R', y_label ='flow')
    single_plot(x = R, y = v_av, var_list = density_list, var_name = 'density', \
                title = 'R influence', x_label = 'R', y_label ='v average')
    
def single_plot(x, y, var_list, var_name, title, x_label, y_label):
    marks_dots = ['ko', 'rv', 'g^', 'bs', 'mx', 'b>', 'r*'] #fix it - only finite number
    marks_lines = ['k-', 'r-', 'g-', 'b-', 'm-', 'b--', 'r--'] #fix it - only finite number
    labels = []
    for i in range(len(var_list)):
        labels.append(var_name + ' = ' + str(var_list[i]))

    #plt.title(title)
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    for i in range(len(var_list)):
        plt.plot(x[i], y[i], marks_dots[i], label = labels[i], markersize=2)
    plt.legend()
    plt.savefig(title + ".tiff", dpi=300, format="tiff")
    plt.show()
    
def draw_them_all(num_lanes, num_places, r_hdv_list, P_lc, time_steady, time_iters, density_steps, R, density, max_platoon_size):
    draw_fundamental_diagram(num_lanes, num_places, density_steps, P_lc, time_steady, time_iters, 
                             r_hdv_list, max_platoon_size, R)
    #draw_R_inf(num_lanes, num_places, P_lc, time_steady, time_iters, R_steps, density_list)

In [ ]:
import time
import matplotlib.pyplot as plt
import numpy as np
import matplotlib as mpl
#from tqdm import tqdm
from tqdm.notebook import tqdm

%matplotlib inline
%config InlineBackend.figure_format = 'svg' 

mpl.rcParams['font.family'] = 'serif'
mpl.rcParams['font.weight'] = 'ultralight'
mpl.rcParams['font.serif'] = 'Times New Roman'
mpl.rcParams['font.size'] = 12
mpl.rcParams['font.style'] = 'normal'
mpl.rcParams['mathtext.default'] = 'regular'

t0 = time.perf_counter()

#draw_fundamental_diagram(num_lanes = 1, num_places = 1000, density_steps = 75, P_lc = 0.0, time_steady = 1000, 
#                         time_iters = 100, r_hdv_list = [0.0, 0.25, 0.5, 0.75, 1.0], max_platoon_size = 6, R = 0.0)

draw_fundamental_diagram_1(num_lanes = 2, num_places = 100, density_steps = 700, P_lc = 0.75, time_steady = 1000, 
                         time_iters = 1000, r_hdv = 1.0, max_platoon_size_list = [0], R = 0.0)

t1 = time.perf_counter() - t0
print("Program running time is t =", t1, "s")

In [ ]:
#draw_fundamental_diagram_1(num_lanes = 1, num_places = 1000, density_steps = 100, P_lc = 0.0, time_steady = 5000, 
#                         time_iters = 4000, r_hdv = 0.0, max_platoon_size_list = [0, 1, 5, 10, 100], R = 0.0)